The most common and valuable composition is taking:

``PromptTemplate`` / ``ChatPromptTemplate`` -> ``LLM`` / ``ChatModel`` -> ``OutputParser``

Almost any other chains you build will use this building block.

## PromptTemplate + LLM

The simplest composition is just combining a prompt and model to create a chain that takes user input, adds it to a prompt, passes it to a model, and returns the raw model output.

Note, you can mix and match PromptTemplate/ChatPromptTemplates and LLMs/ChatModels as you like here.

In [1]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI

prompt = ChatPromptTemplate.from_template("tell me a joke about {foo}")
model = ChatOpenAI()
chain = prompt | model

In [2]:
chain.invoke({"foo": "bears"})

AIMessage(content="Why don't bears wear shoes?\n\nBecause they have bear feet!", additional_kwargs={}, example=False)

Often times we want to attach kwargs that'll be passed to each model call. Here are a few examples of that:

### Attaching Stop Sequences

In [3]:
chain = prompt | model.bind(stop=["\n"])

In [4]:
chain.invoke({"foo": "bears"})

AIMessage(content='Why did the bear never wear shoes?', additional_kwargs={}, example=False)

### Attaching Function Call information

In [5]:
functions = [
    {
        "name": "joke",
        "description": "A joke",
        "parameters": {
            "type": "object",
            "properties": {
                "setup": {"type": "string", "description": "The setup for the joke"},
                "punchline": {
                    "type": "string",
                    "description": "The punchline for the joke",
                },
            },
            "required": ["setup", "punchline"],
        },
    }
]
chain = prompt | model.bind(function_call={"name": "joke"}, functions=functions)

In [6]:
chain.invoke({"foo": "bears"}, config={})

AIMessage(content='', additional_kwargs={'function_call': {'name': 'joke', 'arguments': '{\n  "setup": "Why don\'t bears wear shoes?",\n  "punchline": "Because they have bear feet!"\n}'}}, example=False)

## PromptTemplate + LLM + OutputParser

We can also add in an output parser to easily transform the raw LLM/ChatModel output into a more workable format

In [7]:
from langchain.schema.output_parser import StrOutputParser

chain = prompt | model | StrOutputParser()

Notice that this now returns a string - a much more workable format for downstream tasks

In [8]:
chain.invoke({"foo": "bears"})

"Why don't bears wear shoes?\n\nBecause they have bear feet!"

### Functions Output Parser

When you specify the function to return, you may just want to parse that directly

In [9]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

chain = (
    prompt
    | model.bind(function_call={"name": "joke"}, functions=functions)
    | JsonOutputFunctionsParser()
)

In [10]:
chain.invoke({"foo": "bears"})

{'setup': "Why don't bears like fast food?",
 'punchline': "Because they can't catch it!"}

In [11]:
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser

chain = (
    prompt
    | model.bind(function_call={"name": "joke"}, functions=functions)
    | JsonKeyOutputFunctionsParser(key_name="setup")
)

In [12]:
chain.invoke({"foo": "bears"})

"Why don't bears wear shoes?"

## Simplifying input

To make invocation even simpler, we can add a `RunnableMap` to take care of creating the prompt input dict for us:

In [13]:
from langchain.schema.runnable import RunnableMap, RunnablePassthrough

map_ = RunnableMap(foo=RunnablePassthrough())
chain = (
    map_
    | prompt
    | model.bind(function_call={"name": "joke"}, functions=functions)
    | JsonKeyOutputFunctionsParser(key_name="setup")
)

In [14]:
chain.invoke("bears")

"Why don't bears wear shoes?"

Since we're composing our map with another Runnable, we can even use some syntactic sugar and just use a dict:

In [21]:
chain = (
    {"foo": RunnablePassthrough()}
    | prompt
    | model.bind(function_call={"name": "joke"}, functions=functions)
    | JsonKeyOutputFunctionsParser(key_name="setup")
)

In [22]:
chain.invoke("bears")

"Why don't bears like fast food?"